In [ ]:


import cal
# import imp
# imp.reload(cal)
import os,datetime,copy
from tqdm import tqdm
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False



# 无风险年化收益率
RF=0.015

# 融券年化利率
LR=0.08

# 交易手续费率
STOCK_FEE=0.0002

# 数据路径
PATH_STOCK_DAY="data/stock_day_data/"
PATH_BOND_DAY="data/bond_day_data/"
PATH_CON_PRICE="data/bond_convert_data/"
PATH_BASIC="data/bond_basic_info.csv"


# ## 读取数据获取股债基本信息

# In[3]:


# 基本信息数据
basic_info_data={}
stock_list=[v[0:6] for v in os.listdir(PATH_STOCK_DAY) if v[-4:]==".csv"]
bond_list=set([v[0:6] for v in os.listdir(PATH_BOND_DAY) if v[-4:]==".csv"])
bond_list=bond_list&set([v[0:6] for v in os.listdir(PATH_CON_PRICE) if v[-4:]==".csv"])
bond_list=list(bond_list)
basic_data=pd.read_csv(PATH_BASIC)
print("\neg:\n")
display(basic_data.head(3))
for i in range(len(basic_data)):
    bond_code=str(basic_data["code"][i])
    bond_name=str(basic_data["short_name"][i])
    stock_code=str(basic_data["company_code"][i])[0:6]
    start_date=basic_data["list_date"][i]
    end_date=basic_data["delist_Date"][i]
    market={"上交所":"sh","深交所主板":"sz"}[basic_data["exchange"][i]]
    if stock_code in stock_list and bond_code in bond_list:
        basic_info_data[bond_code]={"bond_code":bond_code,   # 债券代码
                                    "bond_name":bond_name,   # 债券简称
                                    "stock_code":stock_code,   # 股票代码
                                    "start_date":start_date,   # 上市日期
                                    "end_date":end_date,   # 退市日期
                                    "market":market,   # 交易市场
                                   }
print("\neg:\n")
pprint(basic_info_data[list(basic_info_data.keys())[0]])




for code in basic_info_data:
    price_data=[]
    get_data=pd.read_csv(PATH_CON_PRICE+"{}.csv".format(code))
    get_data=get_data.sort_values(by="adjust_date",axis=0,ascending=True)
    for i in range(len(get_data)):
        convert_price=get_data["new_convert_price"][i]
        start_date=max(basic_info_data[code]["start_date"],get_data["adjust_date"][i])
        if i==len(get_data)-1:
            end_date=basic_info_data[code]["end_date"]
        else:
            end_date=get_data["adjust_date"][i+1]
        price_data.append({"convert_price":convert_price,   # 转股价
                           "date":(start_date,end_date),   # 时间区间(<=,<)
                          })
        basic_info_data[code]["convert_price"]=price_data
print("\neg:\n")
display(get_data)
print("\neg:\n")
pprint(basic_info_data[list(basic_info_data.keys())[0]])

start_date="2000-01-01"   # 回测日期 设置
end_date="2022-10-01"
def get_trade_pool(start_date=start_date,end_date=end_date):
    backtest_start_date=start_date
    backtest_end_date=end_date
    benchmark_data=pd.read_csv("data/hs300_day.csv")
    benchmark_data=benchmark_data[benchmark_data["date"]>=backtest_start_date]
    benchmark_data=benchmark_data[benchmark_data["date"]<=backtest_end_date]
    backtest_date_list=list(benchmark_data["date"])
    limit_days=int(0.9*len(backtest_date_list))
    # 交易池
    trade_pool=[]
    for code in basic_info_data:


        trade_pool.append(code)
    print("\n策略池:\n")
    print(trade_pool)
    return trade_pool,backtest_date_list
trade_pool,backtest_date_list=get_trade_pool()


def backtest(trade_pool, backtest_date_list, daybt=0):  # daybt=1为日回测
    backtest_data = {}
    for bond_code in tqdm(trade_pool):
        backtest_start_date=str(cal.strToDate(basic_info_data[bond_code]["start_date"]) + datetime.timedelta(days=90))
        backtest_end_date=str(cal.strToDate(basic_info_data[bond_code]["end_date"]) - datetime.timedelta(days=90))

        stock_code = basic_info_data[bond_code]["stock_code"]
        bond_all_data = pd.read_csv(PATH_BOND_DAY + "{}.csv".format(bond_code))
        stock_all_data = pd.read_csv(PATH_STOCK_DAY + "{}.csv".format(stock_code))
        try:
                        bond_data = bond_all_data[bond_all_data["date"] >= backtest_start_date]
                        bond_data = bond_data[bond_data["date"] <= backtest_end_date]
                        stock_data = stock_all_data[stock_all_data["date"] >= backtest_start_date]
                        stock_data = stock_data[stock_data["date"] <= backtest_end_date]
                        # 以回测第一日为起始日，计算复权修正因子
                        if len(stock_data)>0:
                            stock_data["factor"] = stock_data["factor"] / list(stock_data["factor"])[0]
                            bond_date_list = list(set(list(bond_data["date"])) & set(list(stock_data["date"])))
                            for trade_date in backtest_date_list:
                                # 开始交易
                                if trade_date in bond_date_list:
                                    # 期权有效期T
                                    syear = cal.strToDate(basic_info_data[bond_code]["end_date"]) - cal.strToDate(trade_date)
                                    syear = syear.days / 365
                                    bond_this_day_data = bond_data[bond_data["date"] == trade_date]
                                    stock_this_day_data = stock_data[stock_data["date"] == trade_date]
                                    stock_price = list(stock_this_day_data["close"])[0]
                                    factor_fq = list(stock_this_day_data["factor"])[0]
                                    stock_price_fq = stock_price * factor_fq
                                    bond_price = list(bond_this_day_data["close"])[0]
                                    # 转股价
                                    for cdata in basic_info_data[bond_code]["convert_price"]:
                                        if cdata["date"][0] <= trade_date < cdata["date"][1]:
                                            convert_price = cdata["convert_price"]
                                            break
                                    else:
                                        print(bond_code + " " + trade_date)
                                        if bond_code in backtest_data: del backtest_data[bond_code]
                                        break
                                    # 计算溢价率
                                    pr = bond_price * convert_price / 100 / stock_price - 1
                                    date_data = {"date": trade_date,
                                                 "bond_price": bond_price,
                                                 "stock_price": stock_price,
                                                 "convert_price": convert_price,
                                                 "stock_price_fq": stock_price_fq,
                                                 "factor_fq": factor_fq,
                                                 "pr": pr,
                                                 }

                                    if bond_code not in backtest_data:
                                        # 买入一张可转债
                                        trade_price = list(bond_this_day_data["close"])[0]
                                        date_data["hold_short"] = {"volume": 0}
                                        date_data["cash"] = 0
                                        backtest_data[bond_code] = {"hold_bond": {"cost_price": bond_price, "volume": 1},
                                                                    "date_data": [date_data],
                                                                    "trade_data": []}
                                    else:
                                        last_date_data = backtest_data[bond_code]["date_data"][-1]
                                        last_date = bond_date_list[bond_date_list.index(trade_date) - 1]
                                        last_convert_price = last_date_data["convert_price"]
                                        #
                                        stock_data_sigma = stock_all_data[stock_all_data["date"] < trade_date]
                                        stock_data_sigma = stock_data_sigma[-250:]
                                        stock_data_sigma = stock_data_sigma["close"] * stock_data_sigma["factor"]
                                        stock_sigma = cal.sigma(stock_data_sigma)
                                        #
                                        last_stock_price = last_date_data["stock_price"]
                                        last_bond_price = last_date_data["bond_price"]
                                        # delta_value=cal.delta(last_stock_price,last_convert_price,RF,1,stock_sigma)
                                        delta_value = cal.delta(last_stock_price, last_convert_price, RF, syear, stock_sigma)
                                        # print(delta_value,last_stock_price,last_convert_price,RF,1,stock_sigma)
                                        if "delta" in last_date_data:
                                            # date_data["gamma"]=delta_value-last_date_data["delta"]
                                            date_data["gamma"] = cal.gamma(last_stock_price, last_convert_price, RF, syear, stock_sigma)
                                        date_data["delta"] = delta_value
                                        date_data["sigma"] = stock_sigma
                                        # 原空头持仓
                                        before_short_volume = backtest_data[bond_code]["date_data"][-1]["hold_short"]["volume"]
                                        last_factor_fq = last_date_data["factor_fq"]
                                        before_short_volume = before_short_volume / last_factor_fq * factor_fq
                                        # 计算做空量
                                        short_volume = last_bond_price / last_stock_price * delta_value / last_factor_fq * factor_fq
                                        short_volume = round(short_volume, 1)
                                        if cal.strToDate(trade_date).weekday() == 0 or daybt == 1:
                                            if short_volume > before_short_volume:
                                                # 卖出
                                                sell_volume = round(short_volume - before_short_volume, 1)
                                                # 交易成本：印花税+手续费
                                                charge_money = sell_volume * stock_price * (0.001 + STOCK_FEE)
                                                date_data["cash"] = last_date_data["cash"] + (sell_volume * stock_price - charge_money)
                                                if sell_volume > 0:
                                                    backtest_data[bond_code]["trade_data"].append(
                                                        {"date": trade_date, "type": "sell", "volume": sell_volume, "delta": delta_value})
                                            elif short_volume < before_short_volume:
                                                # 买入
                                                buy_volume = round(before_short_volume - short_volume, 1)
                                                charge_money = buy_volume * stock_price * STOCK_FEE
                                                date_data["cash"] = last_date_data["cash"] - (buy_volume * stock_price + charge_money)
                                                if buy_volume > 0:
                                                    backtest_data[bond_code]["trade_data"].append(
                                                        {"date": trade_date, "type": "buy", "volume": buy_volume, "delta": delta_value})
                                            else:
                                                date_data["cash"] = last_date_data["cash"]
                                            date_data["hold_short"] = {"volume": short_volume}
                                        else:
                                            date_data["cash"] = last_date_data["cash"]
                                            date_data["hold_short"] = {"volume": before_short_volume}
                                        backtest_data[bond_code]["date_data"].append(date_data)

                                    # 计算股债组合净值
                                    this_date_data = backtest_data[bond_code]["date_data"][-1]
                                    if len(backtest_data[bond_code]["date_data"]) > 1:
                                        last_date = backtest_data[bond_code]["date_data"][-2]["date"]
                                        this_date = backtest_data[bond_code]["date_data"][-1]["date"]
                                        time_interval = (cal.strToDate(this_date) - cal.strToDate(last_date)).days
                                        last_date_data = backtest_data[bond_code]["date_data"][-2]
                                        interest_money = last_date_data["stock_price"] * last_date_data["hold_short"]["volume"]
                                        interest_money = interest_money * time_interval * LR / 365
                                        this_date_data["cash"] -= interest_money
                                    total_value = bond_price * backtest_data[bond_code]["hold_bond"]["volume"]
                                    total_value += this_date_data["cash"]
                                    total_value -= (
                                                this_date_data["hold_short"]["volume"] * this_date_data["stock_price"] * (1 + STOCK_FEE))
                                    date_data["total_value"] = total_value
                                    date_data["net_value"] = total_value / backtest_data[bond_code]["date_data"][0]["total_value"]
                                    date_data["stock_net_value"] = stock_price_fq / backtest_data[bond_code]["date_data"][0][
                                        "stock_price_fq"]
                                    date_data["bond_net_value"] = bond_price / backtest_data[bond_code]["date_data"][0]["bond_price"]
                                else:
                                    if bond_code in backtest_data:
                                        backtest_data[bond_code]["date_data"].append(
                                            copy.deepcopy(backtest_data[bond_code]["date_data"][-1]))
        except:
            print('data error')
    print("策略回测结束")
    return backtest_data


backtest_data = backtest(trade_pool, backtest_date_list)

dir_path="all_data/output/"
mergedata=pd.DataFrame(pd.to_datetime(backtest_date_list))
mergedata.columns=['date']
mergedata=mergedata[mergedata['date']>'2018-1-1']

if not os.path.exists(dir_path):os.makedirs(dir_path)
for i in tqdm(backtest_data):
    pf=cal.listDictToDf(backtest_data[i]['date_data'])
    pf[i]=pf['net_value']/pf['net_value'].shift(1)-1
    pf=pf[pf['date']>'2018-1-1']
    pf_output=pf.astype(str).drop_duplicates()
    subdata=pf[['date',i]]
    subdata['date'] = pd.to_datetime(subdata['date'])

    subdata=subdata.drop_duplicates()
    mergedata=mergedata.merge(subdata,on='date',how='left')
    mergedata=mergedata.drop_duplicates('date')
    file_path=dir_path+"{}.csv".format(i)
    #pf_output.to_csv(file_path,index=False,encoding="utf_8_sig")

mergedata=mergedata.sort_values('date')

mergedata.to_excel('all_net_ret.xlsx')


eg:



,id,code,short_name,short_name_spelling,full_name,list_status_id,list_status,issuer,company_code,issue_start_date,...,bond_type,bond_form_id,bond_form,list_date,delist_Date,interest_begin_date,maturity_date,interest_date,last_cash_date,cash_comment
0,1,125301,丝绸转债,SCZZ,吴江丝绸股份有限公司可转换公司债券,301006,终止上市,江苏东方盛虹股份有限公司,000301.XSHE,1998-08-28,...,可转债,704001,记账式,1998-09-15,2003-08-28,1998-08-28,2003-08-28,08-28,2003-08-28,NaN
1,2,125302,茂炼转债,MLZZ,中国石化茂名炼油化工股份有限公司可转换公司债券,301006,终止上市,中国石化茂名炼油化工股份有限公司,NaN,1999-07-28,...,可转债,704001,记账式,1999-08-17,2004-07-28,1999-07-28,2004-07-28,07-28,2004-07-28,NaN
2,3,125898,鞍钢转债,agzz,鞍钢新轧钢股份有限公司可转换公司债券,301006,终止上市,鞍钢股份有限公司,000898.XSHE,2000-03-14,...,可转债,704001,记账式,2000-04-17,2005-03-14,2000-03-14,2005-03-14,存续期内每年3月14日,2005-03-14,NaN



eg:

{'bond_code': '123035',
 'bond_name': '利德转债',
 'end_date': '2025-11-14',
 'market': 'sz',
 'start_date': '2019-12-09',
 'stock_code': '300296'}

eg:



,id,code,name,pub_date,adjust_date,new_convert_price,adjust_reason
0,5115,123151,康泰医学系统(秦皇岛)股份有限公司向不特定对象发行可转换公司债券,2022-06-28,2022-06-28,28.22,初始转股价



eg:

{'bond_code': '123035',
 'bond_name': '利德转债',
 'convert_price': [{'convert_price': 7.04,
                    'date': ('2019-12-09', '2020-05-15')},
                   {'convert_price': 6.98,
                    'date': ('2020-05-15', '2021-05-24')},
                   {'convert_price': 6.96,
                    'date': ('2021-05-24', '2022-05-17')},
                   {'convert_price': 6.91,
                    'date': ('2022-05-17', '2025-11-14')}],
 'end_date': '2025-11-14',
 'market': 'sz',
 'start_date': '2019-12-09',
 'stock_code': '300296'}

策略池:

['123035', '100087', '126301', '125729', '100726', '100236', '100795', '100117', '110001', '125959', '110037', '110317', '110874', '125932', '125937', '125822', '125488', '110036', '110010', '110325', '110423', '110398', '125024', '110488', '110232', '110021', '128031', '125717', '110078', '125960', '110026', '110567', '110971', '110227', '125709', '110598', '125572', '110368', '125528', '110002', '110003', '110004', '110005', '11

...

  5%|████▏                                                                            | 36/699 [00:39<16:40,  1.51s/it]

125572 2008-04-29


  5%|████▍                                                                            | 38/699 [00:40<11:23,  1.03s/it]

125528 2008-08-04


 17%|█████████████▍                                                                  | 117/699 [03:34<26:58,  2.78s/it]